In [ ]:
%%capture
!pip install pycaret[full] --ignore-installed

In [ ]:
## The magic four
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

#garbage collection (clear up some RAM)
import gc

#pycaret
from pycaret.classification import *

#cuML
import cudf

%matplotlib inline

In [ ]:
#this is an aesthetic choice and just removes the many warnings that some functions and comands produce
#it helps significantly declutter the workbook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = cudf.read_csv('../input/tabular-playground-series-nov-2021/train.csv', index_col = 'id')
test = cudf.read_csv('../input/tabular-playground-series-nov-2021/test.csv', index_col = 'id')

In [ ]:
train.info()

In [ ]:
#reduce memory usage

target = train.pop('target').astype(np.int8)
small_train = train.astype(np.float32)
small_train = small_train.join(target)
#assert(small_train.columns == train.columns)
small_train.info()

In [ ]:
clf = setup(data = small_train.to_pandas(), 
            target = 'target',
            data_split_stratify = True,
            #feature_selection = True,
            normalize = True,
            normalize_method = 'robust',
            use_gpu = True,
            fold = 5,
            n_jobs = -1, 
            silent = True
           )
gc.collect()

In [ ]:
#compare_models(sort = 'AUC', exclude = ['gbc','ada','dt','svm','et'])

In [ ]:
lr = create_model('lr')
predict_model(lr);

In [ ]:
final_lr = finalize_model(lr)
predict_model(final_lr);

In [ ]:
tuned_lr = tune_model(lr, optimize = 'AUC', choose_better = True)
predict_model(tuned_lr);

In [ ]:
final_tuned_lr = finalize_model(tuned_lr)
predict_model(final_tuned_lr);

In [ ]:
unseen_predictions_lr = predict_model(final_lr, data=test.to_pandas(), raw_score = True, round = 10)
unseen_predictions_lr.head()

In [ ]:
assert(len(test.index)==len(unseen_predictions_lr))

sub = pd.DataFrame(list(zip(test.to_pandas().index, unseen_predictions_lr.Score_1)),columns = ['id', 'target'])

sub.to_csv('submission.csv', index = False)

print(sub)

In [ ]:
N = 3
top = compare_models(sort = 'AUC', exclude = ['gbc','ada','dt','svm','et'], n_select = N)

In [ ]:
blend = blend_models(top)
predict_model(blend);

In [ ]:
final_blend = finalize_model(blend)
predict_model(final_blend);

In [ ]:
#stack = stack_models(top)
#predict_model(stack);

In [ ]:
tuned_top = [tune_model(i, optimize = 'AUC', choose_better = True) for i in top]
tuned_blend = blend_models(tuned_top)
predict_model(tuned_blend);

In [ ]:
final_tuned_blend = finalize_model(tuned_blend)
predict_model(final_tuned_blend);

In [ ]:
unseen_predictions_blend = predict_model(final_blend, data=test.to_pandas(), raw_score = True, round = 10)
unseen_predictions_blend.head()

In [ ]:
assert(len(test.index)==len(unseen_predictions_blend))

sub = pd.DataFrame(list(zip(test.to_pandas().index, unseen_predictions_blend.Score_1)),columns = ['id', 'target'])

sub.to_csv('submission.csv', index = False)

print(sub)